In [59]:
import unicodedata

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [60]:
def quitar_tildes(texto):
    # Normalizar el texto para descomponer los caracteres acentuados
    normalizado = unicodedata.normalize('NFD', texto)
    # Filtrar los caracteres diacríticos combinados
    texto_limpio = ''.join(c for c in normalizado if not unicodedata.combining(c))
    return texto_limpio

In [61]:
df = pd.read_excel("../data/raw/Reporte SSMOdigital Convenios.xlsx")
# Limpia nombres de convenios
df["NomRevisor"] = df["NomRevisor"].str.split().str.join(" ")
df["NomRevisor"] = df["NomRevisor"].apply(quitar_tildes)

# Limpia nombres de largo 4
nombres_largo_4 = df["NomRevisor"].str.split().str.len() == 4
nombres_convenios_largo_4_limpios = df[nombres_largo_4]["NomRevisor"].str.split()
nombres_convenios_largo_4_limpios = (
    nombres_convenios_largo_4_limpios.str[0]
    + " "
    + nombres_convenios_largo_4_limpios.str[2]
    + " "
    + nombres_convenios_largo_4_limpios.str[3]
)

df.loc[nombres_largo_4, "NomRevisor"] = nombres_convenios_largo_4_limpios

## Identificar la procedencia de cada uno de los movimientos de un convenio

En este apartado se quiere identificar la unidad en donde se ha realizado un movimiento a cada
convenio.


En este apartado se calcula la cantidad de tiempo utilizado entre cada uno de los movimiento de
un convenio.


In [62]:
# Deja solamente los convenios y los ordena por su numero y fecha de accion
df_convenios = df.query("Categoria == 'Convenio con Entidades Públicas'").copy()
df_convenios = df_convenios.sort_values(["NumInterno", "FechaHistorico"])

# Calcula el tiempo entre 2 acciones distintas para cada convenio
df_convenios["tiempo_utilizado"] = df_convenios.groupby("NumInterno")["FechaHistorico"].diff()

# Calcula el tiempo total utilizado para cada convenio
tiempo_total_por_convenio = df_convenios.groupby("NumInterno")["FechaHistorico"].apply(
    lambda x: x.max() - x.min()
)

En este apartado se identifica cada una de las personas en el SSMO, y se formatea su nombre para
unirlo a los movimientos de los convenios.


In [63]:
df_personas = pd.read_excel(
    "../data/raw/Plano DSSMO dic 2023.xlsx",
    usecols=[
        "Nombre Funcionario",
        "Código Unidad",
        "Descripción Unidad",
        "Código Unidad 2",
        "Descripción Unidad 2",
    ],
).drop_duplicates()

Los nombres de las personas estan escritos en distintos formatos en las bases de datos.

- En los convenios estan escritos como: PRIMER NOMBRE - PRIMER APELLIDO - SEGUNDO APELLIDO.
- En el registro de personas estan en el formato: PRIMER APELLIDO - SEGUNDO APELLIDO - PRIMER NOMBRE - SEGUNDO NOMBRE.

Por lo tanto, es necesario llegar a un formato similar.


In [64]:
# Separa los nombres de los funcionarios para llevarlos al formato
nombres_separados = df_personas["Nombre Funcionario"].apply(quitar_tildes)
nombres_separados = nombres_separados.str.split()
df_personas["nombre_formateado"] = (
    nombres_separados.str[2] + " " + nombres_separados.str[0] + " " + nombres_separados.str[1]
)
# Filtra columnas necesarias
# df_personas = df_personas[["nombre_formateado", "Nombre Funcionario", "Descripción Unidad"]]

In [65]:
# Une ambas bases de datos segun el nombre del funcionario
convenios_con_procedencia = df_convenios.merge(
    df_personas, how="left", left_on="NomRevisor", right_on="nombre_formateado"
)